In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv

feature, label = [], []
with open('/content/drive/My Drive/final_512_max.csv', newline='') as file:
  reader = csv.reader(file, delimiter=',')
  next(reader, None)
  for row in reader:
    feature.append([float(x) for x in row[:-1]])
    label.append(int(row[-1])-1)

In [3]:
drive.flush_and_unmount()

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical

min_feat = 931 

X, y = np.array(feature), np.array(label)
k_fold = KFold(shuffle=True)
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

total_score = 0.0

for train, test in k_fold.split(X, y):
  X_train = X[train]
  y_train = y[train]
  X_test = X[test]
  y_test = y[test]

  model = keras.Sequential()
  model.add(layers.LSTM(300, input_shape=(None, min_feat))) # need to change parameter
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(5, activation='softmax')) # need to specify activation function
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
  model.fit(X_train.reshape(len(X_train), 1, min_feat), to_categorical(y_train), epochs=100)

  scores = model.evaluate(X_test.reshape(len(X_test), 1, min_feat), to_categorical(y_test), verbose=0)
  total_score = total_score + scores[1]
  print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/100
123/123 [==============================] - 4s 16ms/step - loss: 1.5845 - acc: 0.3052
Epoch 2/100
123/123 [==============================] - 2s 15ms/step - loss: 1.3457 - acc: 0.4645
Epoch 3/100
123/123 [==============================] - 2s 15ms/step - loss: 1.1621 - acc: 0.5789
Epoch 4/100
123/123 [==============================] - 2s 16ms/step - loss: 1.1099 - acc: 0.5979
Epoch 5/100
123/123 [==============================] - 2s 15ms/step - loss: 1.1064 - acc: 0.6110
Epoch 6/100
123/123 [==============================] - 2s 15ms/step - loss: 1.0964 - acc: 0.6195
Epoch 7/100
123/123 [==============================] - 2s 15ms/step - loss: 1.0607 - acc: 0.6190
Epoch 8/100
123/123 [==============================] - 2s 16ms/step - loss: 1.0078 - acc: 0.6455
Epoch 9/100
123/123 [==============================] - 2s 16ms/step - loss: 1.0128 - acc: 0.6596
Epoch 10/100
123/123 [==============================] - 2s 15ms/step - loss: 0.9990 - acc: 0.6570
Epoch 11/100
123/123 [=======

In [5]:
print("Accuracy: %.2f%%" % ((total_score/5)*100))

Accuracy: 90.95%
